In [4]:
from urllib.parse import urljoin

url = "https://www.jjlim.net/html/a.html"
print(urljoin(url, 'b.html'))
print(urljoin(url, 'http://ak.com/b.html'))

https://www.jjlim.net/html/b.html
http://ak.com/b.html


In [ ]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from urllib.request import urlretrieve
from os import makedirs
import os.path, time, re

url = "https://docs.python.org/3.6/library/"

proc_files = {}

def enum_links(html, base):
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.select("link[rel='stylesheet']")
    links += soup.select('a[href]')
    result = []
    
    for a in links:
        href = a.attrs['href']
        url = urljoin(base, href) # 절대경로로 바꿔주기 위해서 처음 url에다 href에 있는 뒤에 주소를 합침
        result.append(url)
        
    return result

def download_file(url):
    test_url = urlparse(url)
    savepath = './' + test_url.netloc + test_url.path
    
    if re.search(r'/$', savepath):
        savepath += 'index.html'
    # savepath: ./docs.python.org/3.6/library/index.html        

    savedir = os.path.dirname(savepath)
    # savedir: ./docs.python.org/3.6/library 
    
    # savepath 경로와 파일이 이미 있을경우 다른 작업 없이 savepath를 리턴
    if os.path.exists(savepath):
        return savepath
    
    # 다운로드할때 폴더(savedir)이름이 이미 생성 되었는지 확인. 없으면 만들기.
    if not os.path.exists(savedir):
        print('mkdir=', savedir)
        makedirs(savedir)
    
    try:
        # 다운로드 됐을경우 다운로드 완료를 알리고, savapath 경로와 파일 이름으로 다운로드 파일 생성
        # savepath를 리턴
        print('download=', url)
        urlretrieve(url, savepath)
        time.sleep(1)
        return savepath
    
    # 오류 났을경우 다운로드 실패라고 표시해라
    except:
        print('download failed: ', url)
        return None

def analyze_html(url, root_url):
    savepath = download_file(url)
    
    if savepath is None:
        return
    
    # 이미 분석한(proc_files에 있으면) 파일이면 함수실행 나가기
    if savepath in proc_files:
        return
    
    # 분석하지 않은 savepath 이므로 딕셔너리로 {경로:True}로 저장
    proc_files[savepath] = True
    print('analyze_html=', url)
    
    # savepath의 index.html을 읽기
    with open(savepath, encoding='utf-8') as f:
        html = f.read()
        links = enum_links(html, url) # css 파일이나 a href에 있는 경로를 절대 경로로 만든 리스트
        
        
        for link_url in links:
            if link_url.find(root_url) != 0:
                if not re.search(r'.css$', link_url):
                    continue
            
            # html이나 htm인 확장자면 재귀 함수로 분석을 다시 시작
            if re.search(r'.(html|html)$', link_url):
                analyze_html(link_url, root_url)
                continue
            
            # 다운로드 실행
            download_file(link_url)

#실행
analyze_html(url, url)
        








analyze_html= https://docs.python.org/3.6/library/asyncio-task.html
download= https://docs.python.org/3.6/library/asyncio-protocol.html
analyze_html= https://docs.python.org/3.6/library/asyncio-protocol.html
download= https://docs.python.org/3.6/library/asyncio-stream.html
analyze_html= https://docs.python.org/3.6/library/asyncio-stream.html
download= https://docs.python.org/3.6/library/asyncio-subprocess.html
analyze_html= https://docs.python.org/3.6/library/asyncio-subprocess.html
download= https://docs.python.org/3.6/library/asyncio-sync.html
analyze_html= https://docs.python.org/3.6/library/asyncio-sync.html
download= https://docs.python.org/3.6/library/asyncio-queue.html
analyze_html= https://docs.python.org/3.6/library/asyncio-queue.html
download= https://docs.python.org/3.6/library/asyncio-dev.html
analyze_html= https://docs.python.org/3.6/library/asyncio-dev.html
download= https://docs.python.org/3.6/library/asyncore.html
analyze_html= https://docs.python.org/3.6/library/asynco

In [4]:
from bs4 import BeautifulSoup
import urllib.request as ureq
import datetime

url = "https://finance.naver.com/marketindex/"
html = ureq.urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

price = soup.select_one('div.head_info > span.value').string
change = soup.select_one('div.head_info > span.change').string
blind = soup.select_one('div.head_info > span.blind').string
exchange = datetime.date.today()
folder = './exrate/'
file_n = folder + exchange.strftime('%Y-%m-%d') + '.txt'

with open(file_n, 'w', encoding='utf-8') as g:
    g.write('환율 현재 가격은 {}원이고 변동 가격은 {}원으로 {} 입니다.'.format(price,change,blind))